<a href="https://colab.research.google.com/github/Swa-hub/Malaria-Diagnosis/blob/main/MALARIA_DIAGNOSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tdfs
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import InputLayer,Dense,Flatten,Conv2D,MaxPool2D,BatchNormalization,Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy,TruePositives,TrueNegatives,FalsePositives,FalseNegatives,Recall,Precision,AUC
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sns
from google.colab import drive

In [ ]:
dataset,dataset_info=tdfs.load('malaria',shuffle_files=True,as_supervised=True,with_info=True,split=['train'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
for data in dataset[0].take(1):
  print(data)

In [ ]:
dataset_info

In [ ]:
def splits(dataset,train_ratio,val_ratio,test_ratio):

  dataset_size=len(list(dataset.as_numpy_iterator()))

  train_dataset=dataset.take(int(train_ratio*dataset_size))

  skip_val_dataset=dataset.skip(int(train_ratio*dataset_size))
  val_dataset=skip_val_dataset.take(int(val_ratio*dataset_size))

  test_dataset=skip_val_dataset.skip(int(test_ratio*dataset_size))

  return train_dataset,val_dataset,test_dataset

In [ ]:
train_ratio=0.8
val_ratio=0.1
test_ratio=0.1


train_dataset,val_dataset,test_dataset=splits(dataset[0],train_ratio,val_ratio,test_ratio)

print(list(train_dataset.take(1).as_numpy_iterator()),
      list(val_dataset.take(1).as_numpy_iterator()),
      list(test_dataset.take(1).as_numpy_iterator()))


**DATASET VISUALIZATION**

In [ ]:
for i,(image,label) in enumerate(train_dataset.take(16)):
  ax=plt.subplot(4,4,i+1)
  plt.imshow(image)
  plt.title(dataset_info.features['label'].int2str(label))
  plt.axis('off')

**DATA PREPROCESSING**

In [ ]:
# This contains 2 steps
# 1)Resizing the image
# 2)Performing standardization or normalization

In [ ]:
im_size=224

def resize_rescale(image,label):
  return tf.image.resize(image,(im_size,im_size))/255.0,label


In [ ]:
train_dataset=train_dataset.map(resize_rescale)

val_dataset=val_dataset.map(resize_rescale)

test_dataset=test_dataset.map(resize_rescale)

In [ ]:
for image,label in train_dataset.take(1):
  print(image,label)

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
 

In [ ]:
for image,label in test_dataset.take(1):
  p=image
  print(p.shape)

(224, 224, 3)


In [ ]:
BATCH_SIZE=32

train_dataset=train_dataset.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset=val_dataset.shuffle(buffer_size = 8 , reshuffle_each_iteration = True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

#test_dataset=test_dataset.shuffle(buffer_size = 8 , reshuffle_each_iteration = True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [ ]:
val_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [ ]:
test_dataset

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

##MODEL CREATION

##Sequential API

In [ ]:
lenet_model = tf.keras.Sequential([  #sequential api

    InputLayer(input_shape=(im_size,im_size,3)),

    Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Flatten(),

    Dense(100,activation='relu'),
    BatchNormalization(),

    Dense(10,activation='relu'),
    BatchNormalization(),

    Dense(1,activation='sigmoid')
])

In [ ]:
lenet_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 222, 222, 6)         │             168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_14               │ (None, 222, 222, 6)         │              24 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 111, 111, 6)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 109, 109, 16)        │             880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 109, 109, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 54, 54, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 46656)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 100)                 │       4,665,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 10)                  │           1,010 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ (None, 10)                  │              40 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

##Functional API

In [ ]:
func_input=Input(shape=(im_size,im_size,3) , name = 'Input Image')

x=Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu')(func_input)
x=BatchNormalization()(x)
x = MaxPool2D(pool_size=2,strides=2)(x)

x = Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu')(x)
x = BatchNormalization()(x)
output = MaxPool2D(pool_size=2,strides=2)(x)

feature_extractor_model=Model(func_input,output,name='Feature_Extractor')

feature_extractor_model.summary()

Model: "Feature_Extractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 222, 222, 6)         │             168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ (None, 222, 222, 6)         │              24 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 111, 111, 6)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 109, 109, 16)        │             880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ (None, 109, 109, 16)        │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 54, 54, 16)          │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,136 (4.44 KB)

 Trainable params: 1,092 (4.27 KB)

 Non-trainable params: 44 (176.00 B)

In [ ]:
func_input=Input(shape=(im_size,im_size,3) , name = 'Input Image')

x=feature_extractor_model(func_input)

x = Flatten()(x)

x = Dense(100,activation='relu')(x)
x = BatchNormalization()(x)

x = Dense(10,activation='relu')(x)
x = BatchNormalization()(x)

func_output = Dense(1,activation='sigmoid')(x)

lenet_model_func=Model(func_input,func_output,name='Lenet_Model')

lenet_model_func.summary()

Model: "Lenet_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Feature_Extractor (Functional)       │ (None, 54, 54, 16)          │           1,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 46656)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 100)                 │       4,665,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 10)                  │           1,010 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_21               │ (None, 10)                  │              40 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [ ]:
feature_extractor_sequential_model = tf.keras.Sequential([

    InputLayer(input_shape=(im_size,im_size,3)),

    Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2)])


##Callable Model

In [ ]:
func_input=Input(shape=(im_size,im_size,3) , name = 'Input Image')

x=feature_extractor_sequential_model(func_input)

x = Flatten()(x)

x = Dense(100,activation='relu')(x)
x = BatchNormalization()(x)

x = Dense(10,activation='relu')(x)
x = BatchNormalization()(x)

func_output = Dense(1,activation='sigmoid')(x)

lenet_model_func=Model(func_input,func_output,name='Lenet_Model')

lenet_model_func.summary()

Model: "Lenet_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_3 (Sequential)            │ (None, 54, 54, 16)          │           1,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 46656)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 100)                 │       4,665,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (None, 100)                 │             400 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 10)                  │           1,010 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, 10)                  │              40 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

##Model Subclassing

In [ ]:
class FeatureExtractor(Layer):
  def __init__(self,filters,kernel_size,strides,padding,activation,pool_size):
    super(FeatureExtractor,self).__init__()

    self.conv_1 = Conv2D(filters=filters*2,kernel_size=kernel_size,strides=strides,padding=padding,activation=activation)
    self.batch_1 = BatchNormalization()
    self.pool_1 = MaxPool2D(pool_size=pool_size,strides=2*strides)

    self.conv_2 = Conv2D(filters=filters*2,kernel_size=kernel_size,strides=strides,padding=padding,activation=activation)
    self.batch_2 = BatchNormalization()
    self.pool_2 = MaxPool2D(pool_size=pool_size,strides=2*strides)

  def call(self,x,training):

    x = tf.keras.layers.Reshape((224, 224, 3))(x)

    x=self.conv_1(x)
    x=self.batch_1(x)
    x=self.pool_1(x)

    x=self.conv_2(x)
    x=self.batch_2(x)
    x=self.pool_2(x)

    print("Shape before reshape:", x.shape)

    x = self.reshape_1(x)  # This is likely where the error occurs
    x = self.flatten_1(x)
    x = self.dense_1(x)

    return x

feature_sub_classed = FeatureExtractor(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu',pool_size=2)

In [ ]:
func_input=Input(shape=(im_size,im_size,3) , name = 'Input Image')

x=feature_sub_classed(func_input,training=False)

x = Flatten()(x)

x = Dense(100,activation='relu')(x)
x = BatchNormalization()(x)

x = Dense(10,activation='relu')(x)
x = BatchNormalization()(x)

func_output = Dense(1,activation='sigmoid')(x)

lenet_model_func=Model(func_input,func_output,name='Lenet_Model')

lenet_model_func.summary()

Shape before reshape: (None, 54, 54, 12)
Shape before reshape: (None, 54, 54, 12)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:1331: UserWarning: Layer 'feature_extractor_13' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: '''FeatureExtractor' object has no attribute 'reshape_1'''
  warnings.warn(


AttributeError: Exception encountered when calling FeatureExtractor.call().

[1mCould not automatically infer the output shape / dtype of 'feature_extractor_13' (of type FeatureExtractor). Either the `FeatureExtractor.call()` method is incorrect, or you need to implement the `FeatureExtractor.compute_output_spec() / compute_output_shape()` method. Error encountered:

'FeatureExtractor' object has no attribute 'reshape_1'[0m

Arguments received by FeatureExtractor.call():
  • args=('<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=None, name=Input Image>',)
  • kwargs={'training': 'False'}

In [ ]:
class LenetModel(Model):
  def __init__(self,filters,kernel_size,strides,padding,activation,pool_size):
    super(LenetModel,self).__init__()

    self.feature_extractor = FeatureExtractor(filters=8,kernel_size=3,strides=1,padding="valid",activation="relu",pool_size=2)

    self.flatten = Flatten()

    self.dense_1 = Dense(100,activation  = 'relu')
    self.batch_1 = BatchNormalization()

    self.dense_2 = Dense(10,activation  = 'relu')
    self.batch_2 = BatchNormalization()

    self.dense_3 = Dense(1,activation = 'sigmoid')

  def call(self,x,training=False):

    x=self.feature_extractor(x,training=training)

    x=self.flatten(x)

    x=self.dense_1(x)
    x=self.batch_1(x)

    x=self.dense_2(x)
    x=self.batch_2(x)

    x=self.dense_3(x)

    return x

lenet_sub_classed=LenetModel(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu',pool_size=2)
lenet_sub_classed(tf.zeros([1,224,224,3]))
lenet_sub_classed.summary()

##Custom Layers

In [ ]:
class NeuralLearnDense(Layer):
  def __init__(self,output_units,activation):
    super(NeuralLearnDense,self).__init__()

    self.output_units = output_units
    self.activation = activation

  def build(self,input_features_shape):
     self.w=self.add_weight(shape=(input_features_shape[-1],self.output_units),initializer = 'random_normal' , trainable = True)
     self.b=self.add_weight(shape=(self.output_units,),initializer = 'random_normal', trainable = True)

  def call(self,input_features):

    pre_output = tf.matmul(input_features,self.w)+self.b

    if (self.activation == 'relu'):
      return tf.nn.relu(pre_output)

    elif (self.activation == 'sigmoid'):
      return tf.math.sigmoid(pre_output)

    else:
      return pre_output

In [ ]:
im_size = 224

lenet_custom_model = tf.keras.Sequential([


    InputLayer(input_shape=(im_size,im_size,3)),

    Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),

    Flatten(),

    NeuralLearnDense(100,activation='relu'),
    BatchNormalization(),

    NeuralLearnDense(10,activation='relu'),
    BatchNormalization(),

    NeuralLearnDense(1,activation='sigmoid')
])

lenet_custom_model.summary()

##ERROR SANCTIONING

In [ ]:
y_true=np.array([0,1,0,0])
y_pred=np.array([0.6,0.51,0.94,0])
bce=tf.keras.losses.BinaryCrossentropy()
bce(y_true,y_pred)

##Model Training

In [ ]:
metrics=[TruePositives(name='tp'),TrueNegatives(name='tn'),FalsePositives(name='fp'),FalseNegatives(name='fn'),
         BinaryAccuracy(name='accuracy'),Precision(name='precision'),Recall(name='recall'),AUC(name='auc')]

In [ ]:
lenet_sub_classed.compile(optimizer=Adam(learning_rate = 0.01),
    loss=BinaryCrossentropy(),
    metrics = metrics
    )

In [ ]:
history=lenet_sub_classed.fit(train_dataset , validation_data = val_dataset  , epochs = 5 , verbose = 1)

In [ ]:
history.history.keys

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'])
plt.show()

**MODEL EVALUATION AND PREDICTION**

In [ ]:
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset

In [ ]:
train_dataset

In [ ]:
#test_dataset=test_dataset.unbatch()
#test_dataset=test_dataset.batch(BATCH_SIZE)
test_dataset.unbatch()

In [ ]:
lenet_sub_classed.evaluate(test_dataset)

#Visualizing Confusion Matrix

In [ ]:
labels=[]
inp=[]
for x,y in test_dataset.as_numpy_iterator():
  labels.append(y)
  inp.append(x)


In [ ]:
for i, image in enumerate(inp):
    print(f"Shape of image {i}: {image.shape}")

In [ ]:
#print(np.array(inp).shape)
#print(np.array(inp)[:,0,...].shape)

In [ ]:
# labels=np.array([i[0] for i in labels])
# print(labels)

In [ ]:
predicted=lenet_model.predict(np.array(inp)[:,0,...])
print(predicted[:,0])

In [ ]:
def parasite_or_not(x):
  x=tf.cast(x,tf.float32)
  if tf.reduce_all(x<0.5):
    return str('P')
  else:
    return str('U')

In [ ]:
parasite_or_not(lenet_model.predict(test_dataset.take(1))[0])

In [ ]:
for i,(image,label) in enumerate(train_dataset.take(9)):
  ax=plt.subplot(3,3,i+1)
  plt.imshow(image[0])
  title_temp=f"{parasite_or_not(label[0])} : {parasite_or_not(lenet_model.predict(image)[0][0])}"
  plt.title(title_temp)
  plt.axis('off')

**SAVING AND LOADING FROM GOOGLE DRIVE**

In [ ]:
drive.mount('/content/drive')

In [ ]:
!cp -r /content/lenet/ /content/drive/MyDrive/lenet_colab/

In [ ]:
!cp -r /content/drive/MyDrive/lenet_colab/ /content/lenet_colab/